# Exploratory Analysis

Group project for the 2019 Data Science Workshop at the University of California, Berkeley.

The project is the Google Analytics Customer Revenue Prediction competition on Kaggle: https://www.kaggle.com/c/ga-customer-revenue-prediction

Group members:

* Andy Vargas (mentor)
* Yuem Park
* Marvin Pohl
* Michael Yeh

In [2]:
import pandas as pd
import math
import numpy as np
import json
import ast
from pandas.io.json import json_normalize
import time
import os
import datetime

Load data:

Note that the data files are too large to upload to GitHub - instead, the directory `./data/` has been added to the .gitignore, which should contain the following files on your local machine, all downloaded from the Kaggle competition website:

* sample_submission_v2.csv
* test_v2.csv
* train_v2.csv

In [ ]:
#def hits_converter(data):
#    return json.loads(json.dumps(ast.literal_eval(data)))

#def customDimensions_converter(data):
#    if data == '[]':
#        return {}
#    else:
#        return hits_converter(data)[0]

#too slow. Faster to load data, then convert columns to appropriate format.
#def load_df1(csv_path='data/train_v2.csv', nrows=None, skiprows=None):
#    conv_dict = {'device': ujson.loads,
#                'geoNetwork': ujson.loads,
#                'totals': ujson.loads,
#                'trafficSource': ujson.loads,
#                'hits': hits_converter,
#                'customDimensions': customDimensions_converter}
#    df = pd.read_csv(csv_path, 
#                     converters=conv_dict, 
#                     dtype={'fullVisitorId': 'str'}, # Important!!
#                     nrows=nrows)
#    return df

In [29]:
def load_df(csv_path, nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [18]:
def date_converter(yyyymmdd):
    #convert date from integer to datetime object
    return pd.to_datetime(yyyymmdd, format='%Y%m%d').date()

In [19]:
def slicer(start_date, num_days, csv_path = 'data/train_v2.csv'):
    #get rows whose dates lie in the num_days-long period beginning on start_date
    #input start_date as an integer YYYYMMDD
    start_date = pd.to_datetime(start_date, format='%Y%m%d').date()
    num_days = datetime.timedelta(num_days)
    reader = pd.read_csv(csv_path, chunksize=100000,
                         converters = {'date': date_converter},
                         dtype={'fullVisitorId': 'str'})
    chunks = []
    i = 0
    for chunk in reader:
        chunk = chunk[(chunk['date'] >= start_date) & (chunk['date'] < start_date + num_days)]
        chunks.append(chunk)
        i+=1
        print(f"Processed {i} chunks.")
    df = pd.concat(chunks)
    return df

In [19]:
target = pd.read_pickle('data/target_8-30-17_raw.pkl')
target.to_csv('data/target_8-30-17_raw.csv')
target=load_df('data/target_8-30-17_raw.csv')
target=target[['fullVisitorId', 'totals.transactionRevenue']]
target.loc[:,'totals.transactionRevenue']=target['totals.transactionRevenue'].astype(float).fillna(value=0)

target.to_pickle('data/target_8-30-17_cleaned.pkl')
#target slice starting 08/30/17, columns: fullVisitorId and totals.transactionRevenue (NaN set to 0.0)

train = pd.read_pickle('data/train_1-28-17_cleaned.pkl')
target = pd.read_pickle('data/target_8-30-17_cleaned.pkl')

In [8]:
common_ids = set(train['fullVisitorId'].unique()).intersection(set(target['fullVisitorId'].unique()))
#visitors in common in training and target sets

target = target[target['fullVisitorId'].isin(common_ids)]
target.to_pickle('data/target.pkl')
#target_8-30-17_cleaned with only IDs in common_ids (final version below)

target = target.groupby('fullVisitorId').sum()
target.groupby('fullVisitorId').sum().loc['1957458976293878100']
target[target['fullVisitorId']=='1957458976293878100']
7.981500e+08 + 1.322940e+10 + 5.390440e+09 +1.672700e+08
#check that it works

target = target.apply(lambda x: np.log(x+1))
target = target.rename(columns={'totals.transactionRevenue': 'logRevenue'})
target.to_pickle('data/target.pkl')
#final target file

In [35]:
def featurize(df, var_name):
    #for each fullVisitorId, count instances of each value of a categorical variable named var_name
    #input is dataframe with only two columns (fullVisitorId and var_name)
    #returns dataframe in which each column is a count of a single value, index = fullVisitorId
    df = df.pivot_table(index='fullVisitorId', columns=var_name, aggfunc=len, fill_value=0)
    df.columns = [f"{var_name}.{col}" for col in df.columns]
    return df

In [41]:
def drop_constant(df):
    #drop constant columns
    for column in df:
        if df[column].nunique(dropna=False) == 1:
            df = df.drop(column, axis=1)
    return df

In [19]:
df

,fullVisitorId,channelGrouping
0,10,Organic Search
1,11,Referral
2,10,Direct
3,12,Organic Search
4,13,Organic Search
5,11,Referral
6,15,Referral
7,14,Organic Search
8,12,Organic Search
9,12,Organic Search


In [21]:
df=pd.read_csv('data/train_v2.csv', dtype={'fullVisitorId': 'str'}, nrows=10)

In [22]:
df.loc[:, 'fullVisitorId']=pd.Series([10,11,10,12,13,11,15,14,12,12])

In [24]:
sdf=df[['channelGrouping', 'date', 'fullVisitorId']]

In [25]:
sdf

,channelGrouping,date,fullVisitorId
0,Organic Search,20171016,10
1,Referral,20171016,11
2,Direct,20171016,10
3,Organic Search,20171016,12
4,Organic Search,20171016,13
5,Referral,20171016,11
6,Referral,20171016,15
7,Organic Search,20171016,14
8,Organic Search,20171016,12
9,Organic Search,20171016,12


In [31]:
featurize(sdf, 'channelGrouping')

,"channelGrouping.('date', 'Direct')","channelGrouping.('date', 'Organic Search')","channelGrouping.('date', 'Referral')"
fullVisitorId,,,
10,1,1,0
11,0,0,2
12,0,3,0
13,0,1,0
14,0,1,0
15,0,0,1


In [2]:
df=pd.read_csv('data/train_v2.csv', dtype={'fullVisitorId': 'str'}, nrows=10)

In [15]:
df.loc[1,'totals']

'{"visits": "1", "hits": "2", "pageviews": "2", "timeOnSite": "28", "sessionQualityDim": "2"}'

In [20]:
train=slicer(20170501, 168)

Processed 1 chunks.
Processed 2 chunks.
Processed 3 chunks.
Processed 4 chunks.
Processed 5 chunks.
Processed 6 chunks.
Processed 7 chunks.
Processed 8 chunks.
Processed 9 chunks.
Processed 10 chunks.
Processed 11 chunks.
Processed 12 chunks.
Processed 13 chunks.
Processed 14 chunks.
Processed 15 chunks.
Processed 16 chunks.
Processed 17 chunks.
Processed 18 chunks.


In [26]:
train['date'].max()

datetime.date(2017, 10, 15)

In [27]:
train.to_csv('data/train_5-1-17.csv')

In [30]:
train_2=load_df('data/train_5-1-17.csv')

Loaded train_5-1-17.csv. Shape: (425146, 60)


In [31]:
train_2.to_pickle('data/train_5-1-17_raw.pkl')

In [32]:
train_2.head()

,Unnamed: 0,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,...,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.keyword
0,10989,Organic Search,"[{'index': '4', 'value': 'South America'}]",2017-06-23,8220376077399595986,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,1498223351,1,1498223351,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10990,Organic Search,"[{'index': '4', 'value': 'Central America'}]",2017-06-23,1000831944282857356,"[{'hitNumber': '1', 'time': '0', 'hour': '8', ...",Not Socially Engaged,1498232484,2,1498232484,...,not available in demo dataset,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10991,Direct,"[{'index': '4', 'value': 'EMEA'}]",2017-06-23,7830153712423772213,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1498237863,1,1498237863,...,not available in demo dataset,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10992,Direct,"[{'index': '4', 'value': 'EMEA'}]",2017-06-23,6641049248090905193,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,1498223514,1,1498223514,...,not available in demo dataset,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10993,Organic Search,"[{'index': '4', 'value': 'North America'}]",2017-06-23,9993372776950522771,"[{'hitNumber': '1', 'time': '0', 'hour': '4', ...",Not Socially Engaged,1498218717,1,1498218717,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_cg=train_2[['fullVisitorId', 'channelGrouping']]

In [5]:
train_cg.head()

,fullVisitorId,channelGrouping
0,8220376077399595986,Organic Search
1,1000831944282857356,Organic Search
2,7830153712423772213,Direct
3,6641049248090905193,Direct
4,9993372776950522771,Organic Search


In [51]:
train_cg=featurize(train_cg,'channelGrouping')

In [39]:
train_cg.shape

(329636, 8)

In [40]:
train_2.columns

Index(['Unnamed: 0', 'channelGrouping', 'customDimensions', 'date',
       'fullVisitorId', 'hits', 'socialEngagementType', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.browserVersion', 'device.browserSize', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.isMobile',
       'device.mobileDeviceBranding', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.flashVersion',
       'device.language', 'device.screenColors', 'device.screenResolution',
       'device.deviceCategory', 'geoNetwork.continent',
       'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.networkDomain', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits',
       'totals.hits', 'totals.pageviews', 'totals.bounces', 

In [42]:
train_3=drop_constant(train_2)

In [43]:
train_3.columns

Index(['Unnamed: 0', 'channelGrouping', 'customDimensions', 'date',
       'fullVisitorId', 'hits', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.operatingSystem', 'device.isMobile',
       'device.deviceCategory', 'geoNetwork.continent',
       'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.networkDomain',
       'totals.hits', 'totals.pageviews', 'totals.bounces', 'totals.newVisits',
       'totals.timeOnSite', 'totals.transactions', 'totals.transactionRevenue',
       'totals.totalTransactionRevenue', 'totals.sessionQualityDim',
       'trafficSource.campaign', 'trafficSource.source',
       'trafficSource.medium', 'trafficSource.isTrueDirect',
       'trafficSource.referralPath', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwords

In [46]:
train_3.shape

(425146, 41)

In [45]:
featurize_cols = [
       'device.browser', 'device.operatingSystem', 'device.isMobile',
       'device.deviceCategory', 'geoNetwork.continent',
       'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.networkDomain',
       'trafficSource.campaign', 'trafficSource.source',
       'trafficSource.medium', 'trafficSource.isTrueDirect',
       'trafficSource.referralPath', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.keyword']

In [52]:
%%time
for col in featurize_cols:
    temp_df = train_3[['fullVisitorId', col]]
    temp_df = featurize(temp_df, col)
    print(f"Finished {col}. Size: {temp_df.shape}.")

Finished device.browser. Size: (329636, 47).
Finished device.operatingSystem. Size: (329636, 18).
Finished device.isMobile. Size: (329636, 2).
Finished device.deviceCategory. Size: (329636, 3).
Finished geoNetwork.continent. Size: (329636, 6).
Finished geoNetwork.subContinent. Size: (329636, 23).
Finished geoNetwork.country. Size: (329636, 216).
Finished geoNetwork.region. Size: (329636, 259).
Finished geoNetwork.metro. Size: (329636, 83).
Finished geoNetwork.city. Size: (329636, 437).


IndexError: index 1375431536 is out of bounds for axis 0 with size 1375431176

In [49]:
train_cg.head()

,channelGrouping.(Other),channelGrouping.Affiliates,channelGrouping.Direct,channelGrouping.Display,channelGrouping.Organic Search,channelGrouping.Paid Search,channelGrouping.Referral,channelGrouping.Social,device.browser.(not set)_x,device.browser.+Simple Browser_x,...,geoNetwork.city.Winnipeg_y,geoNetwork.city.Wrexham_y,geoNetwork.city.Wroclaw_y,geoNetwork.city.Yekaterinburg_y,geoNetwork.city.Yokohama_y,geoNetwork.city.Zagreb_y,geoNetwork.city.Zaragoza_y,geoNetwork.city.Zhongli District_y,geoNetwork.city.Zurich_y,geoNetwork.city.not available in demo dataset_y
fullVisitorId,,,,,,,,,,,,,,,,,,,,,
0000049363351866189,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0000062267706107999,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0000085059828173212,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000085840370633780,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000091131414287111,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [56]:
train_2['geoNetwork.networkDomain'].value_counts()

(not set)            141303
unknown.unknown       60764
comcast.net           13679
rr.com                 7901
verizon.net            7092
                      ...  
spsd.sk.ca                1
uabc.mx                   1
vapn.de                   1
estec.pe                  1
fdlconsultinc.com         1
Name: geoNetwork.networkDomain, Length: 17202, dtype: int64

In [3]:
train_2=pd.read_pickle('data/train_5-1-17_raw.pkl')